<a href="https://colab.research.google.com/github/samiul272/ColabNotebooks/blob/master/nrml.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install -q tensorflow==2.0.0-beta1

     |████████████████████████████████| 87.9MB 36.8MB/s 
     |████████████████████████████████| 3.1MB 31.7MB/s 
     |████████████████████████████████| 501kB 31.4MB/s 


In [0]:
from tensorflow.python.client import device_lib
import tensorflow as tf
import numpy as np 
import pandas as pd
from tqdm import tnrange
from tqdm import tqdm_notebook as tqdm
import seaborn as sns
from numpy.linalg import norm
import pdb
%precision 3

from tensorflow.python.client import device_lib

print(device_lib.list_local_devices())
tf.__version__

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 1375053093535793091
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 8580416851643903713
physical_device_desc: "device: XLA_CPU device"
]


'2.0.0-beta1'

In [0]:
def dist_mat(P1, P2 = None):
    if P2 is not None:
        X1 = tf.tile(tf.expand_dims(tf.reduce_sum(P1**2, axis= -1), axis= -1), [1, tf.shape(P2)[0]])
        X2 = tf.tile(tf.expand_dims(tf.reduce_sum(P2**2, axis= -1), axis= -1), [1, tf.shape(P1)[0]])
        R = P1 @ tf.transpose(P2)
        D = X1 + tf.transpose(X2) - 2*R
        D = tf.sqrt(D)
        return D
    else:
        X1 = tf.tile(tf.expand_dims(tf.reduce_sum(P1**2, axis= -1), axis= -1), [1, tf.shape(P1)[0]])
        R = P1 @ tf.transpose(P1)
        D = X1 + tf.transpose(X1) - 2*R
        D = tf.sqrt(D)
        return D
    
with tf.Session() as sess:
    P1 = tf.random.uniform([4,7])
    P2 = tf.random.uniform([4,7])
    P = dist_mat(P1, P2)
    print(P.eval())

AttributeError: ignored

In [0]:
class NRML:
    def __init__(self, N, K, d):
        self.N = N
        self.K = K
        self.convergence_data = []
        self.W = 0
        self.Wt = tf.Variable(np.zeros(shape = (d,d), dtype = np.float32))
        self.X = tf.placeholder(tf.float32, shape=[N, d])
        self.Y = tf.placeholder(tf.float32, shape=[N, d])

    def fit(self, X, Y, T = 10, plot_convergence = False):
        deltas = []
        W = 0
        with tf.Session() as sess:
            sess.run(tf.initialize_all_variables())
            for i in tqdm(range(T)):
                sess.run(self.step, {self.X: X, self.Y: Y})
                Wt = self.Wt.eval()
                deltas.append(np.mean((Wt - W)**2))
                X = X @ Wt
                Y = Y @ Wt
                W = Wt
        self.convergence_data = deltas
        self.W = W
        return (W, deltas)

    def build_model(self):
        X = self.X
        Y = self.Y      
        
        H1 = self.__diff(X, Y)/(self.K*self.N)
        H2 = self.__diff(Y, X)/(self.K*self.N)
        
        D = X - Y
        H3 = (tf.transpose(D) @ D)/self.N
        
        H = (H1 + H2 - H3)
        e, Wt = tf.linalg.eigh(H)
        Wt = tf.reverse(Wt, axis=[-1])
        self.step = self.Wt.assign(Wt)
        return (X, Y, self.Wt)
    
    def __diff(self, x, y):
        D = dist_mat(y)
        I = tf.argsort(D)
        y_knn_idx = I[:, 1:self.K+1]
        yt = tf.stack([tf.gather(y, i) for i in tf.unstack(y_knn_idx)], 1)
        x = tf.expand_dims(x, 0)
        d = x - yt
        return tf.reduce_sum(tf.transpose(d, perm = [0, 2, 1]) @ d, axis = 0)
    
    def plot_convergence_curve(self):
            data = pd.DataFrame({'Iterations' : range(len(self.convergence_data)), 
                                 '$\Delta$' : self.convergence_data})
            sns.lineplot(x = 'Iterations', 
                         y = '$\Delta$', 
                         data = data)

In [0]:
n1 = [np.random.normal(size=(5,), loc = -100.0 + x*10/4.0, scale = 1.0) for x in range(4)]
n1 = np.array(n1, dtype = np.float32)
n2 = [np.random.normal(size=(5,), loc = -100.0 + x*10/4.0, scale = 1.0) for x in range(4)]
n2 = np.array(n2, dtype = np.float32)

nrml = NRML(N=4, K=4, d=5)
a = nrml.build_model()
a = nrml.fit(n1,n2,10)
print(nrml.W)
# nrml.plot_convergence_curve()

AttributeError: ignored

In [0]:
# Run this cell to mount your Google Drive.
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!pip install pydrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

file_id = '1pwQ3H4aJ8a6yyJHZkTwtjcL4wYWQb7bn'
downloaded = drive.CreateFile({'id': file_id})
print('Downloaded content "{}"'.format(downloaded.GetContentString()))

W0716 17:13:29.253213 140702337783680 __init__.py:44] file_cache is unavailable when using oauth2client >= 4.0.0 or google-auth
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/googleapiclient/discovery_cache/__init__.py", line 36, in autodetect
    from google.appengine.api import memcache
ModuleNotFoundError: No module named 'google.appengine'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/googleapiclient/discovery_cache/file_cache.py", line 33, in <module>
    from oauth2client.contrib.locked_file import LockedFile
ModuleNotFoundError: No module named 'oauth2client.contrib.locked_file'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/googleapiclient/discovery_cache/file_cache.py", line 37, in <module>
    from oauth2client.locked_file import Lock

FileNotDownloadableError: ignored

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
from tensorflow.keras.models import load_model, Model
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import Adam
# !wget https://drive.google.com/open?id=1pwQ3H4aJ8a6yyJHZkTwtjcL4wYWQb7bn

file_id = '1pwQ3H4aJ8a6yyJHZkTwtjcL4wYWQb7bn'
request = drive_service.files().get_media(fileId=file_id)
fh = io.BytesIO()
downloader = MediaIoBaseDownload(fh, request)
done = False
while done is False:
    status, done = downloader.next_chunk()
    print("Download %d%%." % int(status.progress() * 100))
feature_model = load_model('facenet_keras.h5')

NameError: ignored

In [0]:
from PIL import Image
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator as IDG

def get_files_in_dir(directory):
    dirs = []
    for path, subdirs, files in os.walk(directory):
        for name in files:
            dirs.append(os.path.join(path, name))
    return dirs
    
tr_dir = '../NorthEastern_SMILE_dataset/Data/Train/'
class_dirs = [tr_dir + j for j in os.listdir(tr_dir)]
img_dirs = get_files_in_dir(tr_dir)

datagen_x_tr = [np.array(Image.open(img_dirs[np.random.randint(low = 0, high=len(img_dirs)- 2)])) for x in  range(1024)]


data_gen_args = dict(featurewise_center=True,
                     featurewise_std_normalization=True,
                     rotation_range=10,
                     rescale=1./255,
                     width_shift_range=0.1,
                     height_shift_range=0.1,
                     horizontal_flip = True,
                     vertical_flip = False,
                     zoom_range=0.05)

idg = IDG(**data_gen_args)
idg.fit(datagen_x_tr)

image_generator = idg.flow_from_directory(
    tr_dir,
    target_size=(160, 160),
    class_mode='categorical',
    seed=1)

In [0]:
from PIL import Image
import matplotlib.pyplot as plt
import os

def get_files_in_dir(directory):
    dirs = []
    for path, subdirs, files in os.walk(directory):
        for name in files:
            dirs.append(os.path.join(path, name))
    return dirs

def get_image_samples(directory_list, num_samples = 32, img_size = (160,160)):
    C = np.random.choice(class_dirs, size=num_samples)
    I = [np.random.choice(get_files_in_dir(j)) for j in C]
    return np.array([np.array(Image.open(i).resize(img_size)) for i in I])

In [0]:
from tensorflow.keras.utils import Sequence

tr_dir = '../NorthEastern_SMILE_dataset/Data/Train/'
class_dirs = [tr_dir + j for j in os.listdir(tr_dir)]
img_dirs = get_files_in_dir(tr_dir)

class TripletSequence(Sequence):

    def __init__(self, class_dirs, classes, W, model, batch_size=8, Nt=16, K = 4,):
        self.class_dirs, self.classes = class_dirs, classes
        self.batch_size, self.Nt, self.K = batch_size, Nt, K
        self.W = W
        self.model = model

    def __len__(self):
        return 100

    def __getitem__(self, idx):
        pairs = []
        for i in range(self.batch_size):
            x = np.random.choice(class_dirs)
            c = list(self.class_dirs)
            c.remove(x)
            pairs.append(([x], c))
        anchors, positives, negatives = [], [], []
        for c, cbar in pairs:
            anchor_img = get_image_samples(c, num_samples=1)
            pos_img = get_image_samples(c, num_samples=self.Nt)
            neg_img = get_image_samples(cbar, num_samples=self.Nt)
            
            inp = np.concatenate([pos_img, neg_img, anchor_img])
            out = feature_model.predict(inp)
            out = out @ self.W
            anchor, pos, neg = out[-1,:], out[:self.Nt,:], out[self.Nt+1:-1,:]
            
            anchors.append(anchor_img[0])
            positives.append(pos_img[np.argmax(norm(pos - anchor,axis=-1, ord=2))])
            negatives.append(neg_img[np.argmin(norm(neg - anchor,axis=-1, ord=2))])
        return [anchors, positives, negatives]

In [0]:
ALPHA = 0.2

def triplet_loss(x):
    anchor, positive, negative = x

    pos_dist = tf.reduce_sum(tf.square(tf.subtract(anchor, positive)), 1)
    neg_dist = tf.reduce_sum(tf.square(tf.subtract(anchor, negative)), 1)

    basic_loss = tf.add(tf.subtract(pos_dist, neg_dist), ALPHA)
    loss = tf.reduce_mean(tf.maximum(basic_loss, 0.0), 0)

    return loss

def build_facenet_model(embedding_network):
    
    input_shape = embedding_network.input_shape[1:]
    positive_sample = Input(shape=input_shape)
    negative_sample = Input(shape=input_shape)
    anchor_sample = Input(shape=input_shape)


    positive_embedding = embedding_network(positive_sample)
    negative_embedding = embedding_network(negative_sample)
    anchor_embedding = embedding_network(anchor_sample)
    
    loss = triplet_loss([anchor_embedding, positive_embedding, negative_embedding])
    model = Model(inputs=[anchor_sample, positive_sample, negative_sample],
                  outputs=loss)
    model.compile(loss='mean_absolute_error', optimizer=Adam())

    return model

model = build_facenet_model(feature_model)
model.summary()

In [0]:
tr = TripletSequence(class_dirs=class_dirs, classes=class_dirs, W = nrml.W, model=feature_model)
x = tr.__getitem__(1)
